In [126]:
import pandas as pd
import sqlite3

#read csv file by pandas
school = pd.read_csv('ChicagoPublicSchools.csv')
school.index = school.index + 1  #start index from 1

crime = pd.read_csv('ChicagoCrimeData.csv')
crime.index = crime.index + 1

census = pd.read_csv('ChicagoCensusData.csv')
census.index = census.index + 1

#connect to database
conn = sqlite3.connect('ibm_assi.db')
c = conn.cursor()

#drop table if exists
c.execute(" drop table if exists CHICAGO_PUBLIC_SCHOOLS ")
c.execute(" drop table if exists CHICAGO_CRIME_DATA ")
c.execute(" drop table if exists CENSUS_DATA ")

#load datatable into database
school.to_sql('CHICAGO_PUBLIC_SCHOOLS',conn)
crime.to_sql('CHICAGO_CRIME_DATA',conn)
census.to_sql('CENSUS_DATA',conn)

In [143]:
sta = "SELECT * FROM CHICAGO_CRIME_DATA LIMIT 5 ;"
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 1000)
da = pd.read_sql_query(sta, conn)
da.style.hide_index() 

index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
1,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.000000,58.000000,6,1155838.000000,1873050.000000,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
2,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,1112,11,27.000000,23.000000,6,1152206.000000,1906127.000000,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
3,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,0,0,221,2,3.000000,38.000000,6,1177436.000000,1876313.000000,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
4,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,423,4,7.000000,46.000000,6,1194622.000000,1850125.000000,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
5,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,0,0,831,8,15.000000,66.000000,6,1155240.000000,1860661.000000,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [144]:
sta = "SELECT * FROM CENSUS_DATA LIMIT 5;"
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 1000)
da = pd.read_sql_query(sta, conn)
#da.info()
da.style.hide_index()

index,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1,1.000000,Rogers Park,7.700000,23.600000,8.700000,18.200000,27.500000,23939,39.000000
2,2.000000,West Ridge,7.800000,17.200000,8.800000,20.800000,38.500000,23040,46.000000
3,3.000000,Uptown,3.800000,24.000000,8.900000,11.800000,22.200000,35787,20.000000
4,4.000000,Lincoln Square,3.400000,10.900000,8.200000,13.400000,25.500000,37524,17.000000
5,5.000000,North Center,0.300000,7.500000,5.200000,4.500000,26.200000,57123,6.000000


In [145]:
sta = "SELECT * FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 5;"
pd.set_option('display.max_columns',100) 
pd.set_option('display.max_rows', 1000)
da = pd.read_sql_query(sta, conn)
#da.info()
da.style.hide_index() #remove index label

index,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
1,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99.000000,Very Strong,99,Strong,74.000000,Strong,66.000000,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.000000,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.700000,64.400000,0.200000,0.900000,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458000,1915829.428000,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
2,610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54.000000,Strong,66,Strong,74.000000,Very Strong,84.000000,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.700000,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.800000,16.500000,0.700000,1.400000,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985000,1856209.466000,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
3,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61.000000,NDA,NDA,Average,50.000000,Weak,36.000000,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.300000,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.300000,15.500000,-0.900000,-1.000000,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165000,1851012.215000,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
4,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56.000000,Average,44,Average,45.000000,Weak,37.000000,Weak,65,Av

### Problem 1: Find the total number of crimes recorded in the CRIME table.

In [130]:
statement = "   SELECT COUNT(CASE_NUMBER) AS TOTAL_NUMBER_OF_CRIMES \
                FROM CHICAGO_CRIME_DATA"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

TOTAL_NUMBER_OF_CRIMES
533


### Problem 2: List community areas with per capita income less than 11000.

In [131]:
statement = "   SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME \
                FROM CENSUS_DATA \
                WHERE PER_CAPITA_INCOME < 11000;"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3: List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [132]:
statement = "   SELECT CASE_NUMBER, DESCRIPTION  \
                FROM CHICAGO_CRIME_DATA \
                WHERE DESCRIPTION LIKE '%MINOR%'"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### Problem 4: List all kidnapping crimes involving a child?

In [133]:
statement = "   SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION  \
                FROM CHICAGO_CRIME_DATA \
                GROUP BY PRIMARY_TYPE \
                HAVING PRIMARY_TYPE like 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%' "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5: What kinds of crimes were recorded at schools?

In [134]:
statement = "   SELECT distinct(PRIMARY_TYPE) \
                FROM CHICAGO_CRIME_DATA \
                WHERE LOCATION_DESCRIPTION LIKE 'SCHOOL%' "
#statement = " select distinct(LOCATION_DESCRIPTION) FROM CHICAGO_CRIME_DATA" 
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6: List the average safety score for each type of school.

In [135]:
statement =     'SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE \
                 FROM CHICAGO_PUBLIC_SCHOOLS \
                 GROUP BY "Elementary, Middle, or High School";'
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

"Elementary, Middle, or High School",AVERAGE_SAFETY_SCORE
ES,49.520384
HS,49.623529
MS,48.000000


### Problem 7: List 5 community areas with highest % of households below poverty line

In [136]:
statement = "   SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY \
                FROM CENSUS_DATA\
                ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC\
                LIMIT 5"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.500000
Fuller Park,51.200000
Englewood,46.600000
North Lawndale,43.100000
East Garfield Park,42.400000


### Problem 8: Which community area is most crime prone?

In [137]:
statement = "   SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS NUMBER_OF_CRIME \
                FROM CHICAGO_CRIME_DATA \
                GROUP BY COMMUNITY_AREA_NUMBER \
                ORDER BY 2 DESC \
                LIMIT 1"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NUMBER,NUMBER_OF_CRIME
25.000000,43


### Problem 9: Use a sub-query to find the name of the community area with highest hardship index

In [138]:
statement = "   SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, HARDSHIP_INDEX \
                FROM CENSUS_DATA \
                WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA) "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
54.000000,Riverdale,98.000000


### Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [139]:
statement = "   SELECT COMMUNITY_AREA_NAME, MAX(CRIME) AS MOST_NUMBER_OF_CRIMES \
                FROM \
                    (SELECT cd.COMMUNITY_AREA_NAME, count(ccd.COMMUNITY_AREA_NUMBER) as CRIME \
                    FROM CHICAGO_CRIME_DATA ccd,CENSUS_DATA cd \
                    WHERE ccd.COMMUNITY_AREA_NUMBER = cd.COMMUNITY_AREA_NUMBER \
                    GROUP BY cd.COMMUNITY_AREA_NAME) "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,MOST_NUMBER_OF_CRIMES
Austin,43
